In [1]:
# Based on the Git hub page from omoindrot {https://gist.github.com/omoindrot/dedc857cdc0e680dfb1be99762990c9c}

In [2]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os, sys

checkpoints_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/TransferLearning/VGG-CheckPoint'

directory_to_slim = "/Users/sam/App-Setup/anaconda/lib/python3.6/site-packages/tensorflow/models/slim"
# set the path so that import is easy
sys.path.append(directory_to_slim)

In [3]:
%matplotlib inline

import numpy as np
import urllib
import tensorflow as tf
import scipy as spy
from matplotlib import pyplot as plt

from datasets import imagenet
from nets import vgg
from datasets import dataset_utils
from preprocessing import vgg_preprocessing

from tensorflow.python.framework import ops


trainDataFolder = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/coco-animals/train"
testDataFolder = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/coco-animals/train"

In [4]:
def list_images(directory):
    """
    Get all the images and labels in directory/label/*.jpg
    """
    labels = [dirs for dirs in os.listdir(directory) if dirs != ".DS_Store"]
    files_and_labels = []
    print (labels)
    for label in labels:
        for f in os.listdir(os.path.join(directory, label)):
            files_and_labels.append((os.path.join(directory, label, f), label))

    filenames, labels = zip(*files_and_labels)
    filenames = list(filenames)
    labels = list(labels)
    unique_labels = list(set(labels))

    label_to_int = {}
    for i, label in enumerate(unique_labels):
        label_to_int[label] = i

    labels = [label_to_int[l] for l in labels]

    return filenames, labels

### Plot Fuction

In [ ]:
def plot(dataIN, title):
    plt.figure()
    plt.imshow(dataIN.astype(np.uint8))  # Convering into insigned integer
    plt.suptitle(title, fontsize=14, color = 'r')
    plt.axis('off')
    plt.show()

### Resize image:

In [22]:
# VGG takes in a centered.random cropped input of 224. If we take a center crop from 720x1080 image we would miss a 
# major part of the actual image. So here we resize it to 256xScaled_height and then apply crop on the resized
# image

# Reading/Parsing image files using TensorFlow
# Resize the image such that the smaller side is 256 pixels long
def parseImage(inputImagePath, imageLabel):
    imageString = tf.read_file(inputImagePath)
    imageDecoded = tf.image.decode_jpeg(imageString, channels=3)
    image = tf.cast(imageDecoded, tf.float32)
    image2 = image
    
    print (image.get_shape().as_list())
    smallest_side = 256.0
    height, width = tf.shape(image)[0], tf.shape(image)[1]
    height = tf.to_float(height)
    width = tf.to_float(width)

    # Basically the smallest side becomes 256 and larger side = (256*100)/orig_larger_side_size
    scale = tf.cond(tf.greater(height, width),
                    lambda: smallest_side / width,
                    lambda: smallest_side / height)
    new_height = tf.to_int32(height * scale)
    new_width = tf.to_int32(width * scale)

#     print(image.get_shape().as_list())
    imageResized = tf.image.resize_images(image, [new_height, new_width])  # (2)
    return imageResized
#     return imageDecoded

### Preprocessing:

In [26]:
# Here we coulf either use VGG preoprocessing or write a preprocessing
from preprocessing.vgg_preprocessing import _mean_image_subtraction,_R_MEAN, _G_MEAN, _B_MEAN

# RGB mean
VGG_MEAN = [_R_MEAN, _G_MEAN, _B_MEAN]

# print (VGG_MEAN)
def preprocessing(imageIN, imageSize, is_training, is_type="SELF"):
    if is_type == "VGG":
        preprocessedImage = vgg_preprocessing.preprocess_image(imageIN, imageSize, 
                                                               imageSize, is_training=is_training)
    elif is_type == "SELF" and is_training:
        # Crop image
        imageCrop = tf.random_crop(imageIN, [imageSize,imageSize,3])
        # Augment Image by horizontal Flip
        imageFlip = tf.image.random_flip_left_right(imageCrop)
        means = tf.reshape(tf.constant(VGG_MEAN), [1, 1, 3])
        preprocessedImage = imageFlip - means 
        
    elif is_type == "SELF" and not is_training:
        # Crop image
        crop_image = tf.image.resize_image_with_crop_or_pad(imageIN, imageSize, imageSize)
        means = tf.reshape(tf.constant(VGG_MEAN), [1, 1, 3])
        preprocessedImage = imageCrop - means
        
    else:
        raise ValueError('Provide a type Preprocessing')
        
    return imageCrop, imageFlip, means, preprocessedImage

In [30]:
filenames, labels = list_images(trainDataFolder)

ops.reset_default_graph()
sess = tf.Session()

with sess.as_default():
    scale, new_height, new_width, imageResized, image = parseImage(filenames[0], imageLabel=labels[0])
    imageCrop, imageFlip, means, preprocessedImage = preprocessing(imageIN=imageResized, imageSize=224, 
                                                                     is_training=True, is_type="SELF")
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
#         sc, nh, nw, rsimg, img = sess.run([scale, new_height, new_width, resized_image, image])
        imgcrp, imgflp, imgmeans, imgprs= sess.run([imageCrop, imageFlip, means, preprocessedImage])
        print (imgcrp.shape)
        print (imgflp.shape)
        print (imgmeans)
        print (imgprs.shape)
#         print(rsimg.shape)
#         print(img.shape)
#         print (sc)

['bear', 'bird', 'cat', 'dog', 'giraffe', 'horse', 'sheep', 'zebra']
[None, None, 3]
(224, 224, 3)
(224, 224, 3)
[[[ 123.68000031  116.77999878  103.94000244]]]
(224, 224, 3)


In [21]:
256/433 * 640

378.3833718244804